In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from mod5Module import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# capture username and password and CRUD Python module name
username = "aacuser"
password = "2345"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Add in Grazioso Salvare’s logo
image_filename = 'projectlogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.title = 'Dashboard'
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
    html.Div('Valerie Smith CS-340 Project Two', style={'color': 'blue', 'fontSize': 14}),
 
], style={'marginBottom': 50, 'marginTop': 25, 'textAlign': 'center'}),
    html.Div([
        html.A([
            html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())))],
                        href='https://www.snhu.edu'),
        ]),
    html.Hr(),
  
    html.Div(
         
       # interactive filtering options using radio buttons
        dcc.RadioItems(id='radio',
            options=[
                {"label" :"Water Rescue", "value" : "water_rescue"},
                {"label": "Mountain or Wilderness Rescue", "value": "mountain_rescue"},
                {"label": "Disaster or Individual Tracking Rescue", "value": "disaster_rescue"},
                {"label": "Reset", "value": "reset"},
            ],
            
            labelStyle={"display": "inline-block"},
            value = [],          
        ) 
 #   ),
 

 ),   
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        style_table = {'overflowX': 'auto'},
        style_cell = {'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
        'whiteSpace': 'normal'},
        editable= False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
        
        
    ),
    html.Br(),
    html.Hr(),
# setting up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# datatable information with radio buttons    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio', 'value')])
def update_dashboard(input_value):
     
    # query logic for interactive data filtering (radio buttons)
    if input_value == 'water_rescue':
        df = pd.DataFrame.from_records(shelter.read_all({
            "animal_type":"Dog",
            "breed":
                {"$in":["Labrador Retriever Mix","Chesa Bay Retr Mix","Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte": 156}
        }))
    elif input_value == 'mountain_rescue':
        df = pd.DataFrame.from_records(shelter.read_all({
            "animal_type":"Dog",
            "breed":
                {"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}
        }))
    elif input_value == 'disaster_rescue':
        df = pd.DataFrame.from_records(shelter.read_all({
            "animal_type":"Dog",
            "breed":
                {"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}
        }))
    elif input_value == 'reset':
        df = pd.DataFrame.from_records(shelter.read_all({}))
       
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')        
        
    return (data,columns)
    
# update styles
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
# pie chart information
@app.callback(
    Output('graph-id', "children"),
   [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    df_grouped = dff[['breed', '1']].groupby(['breed']).agg('count').reset_index()
    df_grouped = df_grouped.rename(columns = {'1': 'Count'})
    
    fig = px.pie(df_grouped, values='Count', names='breed', title="Animal Types", color_discrete_sequence = px.colors.sequential.RdBu)
    return [
        dcc.Graph(figure=fig)
    ]

@app.callback(
Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # return the icons from page being viewed
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")] +
            # Icons with tool tip popup window with the animal name
            [dl.Marker(position=[row['location_lat'],row['location_long']], children=[
                dl.Tooltip(row['breed']),
                dl.Popup([
                     html.H1("Animal Name"),
                     html.P(row['name'])
                ])
            ]) for index, row in dff.iterrows()]
        )
    ]




app